In [ ]:
# basic stuff first
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')  # hate seeing all those warnings

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px  # might use this later
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# time series stuff - will probably need most of these
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf  # maybe later

# evaluation metrics
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from math import sqrt

# let's see what we're working with
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)

# set some display options
pd.set_option('display.max_columns', 15)  # don't need to see everything

plt.style.use('default')  # keeping it simple
# sns.set_palette("tab10")  # default is fine

print("imports done")


In [ ]:
def generate_campaign_data(days=180, start_date='2024-01-01'):
    """
    Generate some fake campaign data that looks real enough
    """
    
    np.random.seed(42)  # so we get consistent results
    
    dates = pd.date_range(start=start_date, periods=days, freq='D')
    
    # start with base spend - should increase over time as we optimize
    base_spend = np.linspace(1000, 1200, days)  # gradual ramp up
    
    # add weekly patterns - weekends are usually worse
    weekly_pattern = np.sin(2 * np.pi * np.arange(days) / 7) * 0.15 + 1
    
    # some monthly variation too
    monthly_pattern = np.sin(2 * np.pi * np.arange(days) / 30) * 0.1 + 1
    
    # add noise
    noise = np.random.normal(0, 0.08, days)
    
    # calculate daily spend
    daily_spend = base_spend * weekly_pattern * monthly_pattern * (1 + noise)
    daily_spend = np.maximum(daily_spend, 500)  # don't go below $500/day
    
    # impressions based on spend and CPM
    cpm_base = 2.5  # $2.5 CPM
    cpm_variation = np.random.normal(0, 0.3, days)  # CPM varies by day
    impressions = (daily_spend / (cpm_base + cpm_variation)) * 1000
    impressions = np.maximum(impressions, 10000)  # min 10k impressions
    
    # clicks - CTR should improve over time and vary by day of week
    base_ctr = 0.025  # start at 2.5% CTR
    ctr_improvement = np.linspace(0, 0.008, days)  # gets better over time
    
    # weekday effects on CTR
    weekday_ctr = np.array([0.8, 1.0, 1.1, 1.2, 1.15, 0.9, 0.7])  # Mon-Sun
    day_multiplier = np.array([weekday_ctr[date.weekday()] for date in dates])
    
    ctr = (base_ctr + ctr_improvement) * day_multiplier * (1 + np.random.normal(0, 0.1, days))
    clicks = impressions * ctr
    
    # conversions - CVR also improves with optimization
    base_cvr = 0.08  # 8% conversion rate
    cvr_improvement = np.linspace(0, 0.02, days)  # gets better
    cvr = (base_cvr + cvr_improvement) * (1 + np.random.normal(0, 0.15, days))
    conversions = clicks * cvr
    
    # derived metrics
    cpc = daily_spend / clicks
    cpa = daily_spend / conversions
    
    # revenue based on AOV
    avg_order_value = np.random.normal(150, 25, days)
    avg_order_value = np.maximum(avg_order_value, 50)  # min $50 AOV
    revenue = conversions * avg_order_value
    
    # ROAS
    roas = revenue / daily_spend
    
    # put it all together
    data = pd.DataFrame({
        'date': dates,
        'spend': daily_spend,
        'impressions': impressions,
        'clicks': clicks,
        'conversions': conversions,
        'cpc': cpc,
        'cpa': cpa,
        'revenue': revenue,
        'roas': roas,
        'ctr': ctr * 100,  # as percentage
        'cvr': cvr * 100   # as percentage
    })
    
    # clean up the numbers
    data['spend'] = data['spend'].round(2)
    data['impressions'] = data['impressions'].round(0).astype(int)
    data['clicks'] = data['clicks'].round(0).astype(int)
    data['conversions'] = data['conversions'].round(0).astype(int)
    data['revenue'] = data['revenue'].round(2)
    data['cpc'] = data['cpc'].round(3)
    data['cpa'] = data['cpa'].round(2)
    data['roas'] = data['roas'].round(2)
    data['ctr'] = data['ctr'].round(3)
    data['cvr'] = data['cvr'].round(3)
    
    return data

# let's generate the data
print("generating fake campaign data...")
df = generate_campaign_data(days=180, start_date='2024-01-01')

# quick check
print(f"got {len(df)} days of data")
print(f"from {df['date'].min().date()} to {df['date'].max().date()}")
print(f"total spend: ${df['spend'].sum():,.2f}")
print(f"total conversions: {df['conversions'].sum():,.0f}")
print(f"avg ROAS: {df['roas'].mean():.2f}")

# let's see what it looks like
print("\nfirst few rows:")
df.head()


In [ ]:
# basic stats first
print("Summary stats:")
key_metrics = ['spend', 'impressions', 'clicks', 'conversions', 'cpc', 'cpa', 'roas', 'ctr', 'cvr']
summary = df[key_metrics].describe()
print(summary.round(3))

# check correlations
print("\nCorrelations:")
corr_matrix = df[key_metrics].corr()
print(corr_matrix.round(3))

# hmm let's see some quick insights
print(f"\nQuick insights:")
print(f"CPA ranges from ${df['cpa'].min():.2f} to ${df['cpa'].max():.2f}")
print(f"ROAS ranges from {df['roas'].min():.2f} to {df['roas'].max():.2f}")
print(f"CTR avg: {df['ctr'].mean():.3f}%")
print(f"CVR avg: {df['cvr'].mean():.3f}%")


In [ ]:
# let's make some plots to see what's going on
fig, axes = plt.subplots(3, 2, figsize=(15, 16))
fig.suptitle('Campaign Trends', fontsize=14)

# spend over time
axes[0, 0].plot(df['date'], df['spend'])
axes[0, 0].set_title('Daily Spend')
axes[0, 0].set_ylabel('Spend ($)')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# impressions and clicks - let's put both on same plot
ax1 = axes[0, 1]
ax1.plot(df['date'], df['impressions'], color='orange', label='Impressions')
ax2 = ax1.twinx()  # need second y-axis
ax2.plot(df['date'], df['clicks'], color='green', label='Clicks')
ax1.set_title('Impressions vs Clicks')
ax1.set_ylabel('Impressions', color='orange')
ax2.set_ylabel('Clicks', color='green')
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# conversions
axes[1, 0].plot(df['date'], df['conversions'], color='red')
axes[1, 0].set_title('Daily Conversions')
axes[1, 0].set_ylabel('Conversions')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].tick_params(axis='x', rotation=45)

# CPA - this is important
axes[1, 1].plot(df['date'], df['cpa'], color='purple')
axes[1, 1].set_title('CPA Trend')
axes[1, 1].set_ylabel('CPA ($)')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)

# ROAS - super important
axes[2, 0].plot(df['date'], df['roas'], color='brown')
axes[2, 0].set_title('ROAS Over Time')
axes[2, 0].set_ylabel('ROAS')
axes[2, 0].axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Break-even')
axes[2, 0].legend()
axes[2, 0].grid(True, alpha=0.3)
axes[2, 0].tick_params(axis='x', rotation=45)

# CTR and CVR together
axes[2, 1].plot(df['date'], df['ctr'], color='pink', label='CTR (%)')
axes[2, 1].plot(df['date'], df['cvr'], color='gray', label='CVR (%)')
axes[2, 1].set_title('CTR & CVR')
axes[2, 1].set_ylabel('Rate (%)')
axes[2, 1].legend()
axes[2, 1].grid(True, alpha=0.3)
axes[2, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# hmm, let's also look at weekly patterns
df['day_of_week'] = df['date'].dt.day_name()

# let's see if there are clear day-of-week patterns
plt.figure(figsize=(12, 8))

# average by day of week
daily_avg = df.groupby('day_of_week')[['spend', 'conversions', 'roas']].mean()
# reorder to make sense
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_avg = daily_avg.reindex(day_order)

plt.subplot(2, 2, 1)
daily_avg['spend'].plot(kind='bar', color='skyblue')
plt.title('Avg Daily Spend by Day')
plt.ylabel('Spend ($)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
daily_avg['conversions'].plot(kind='bar', color='coral')
plt.title('Avg Conversions by Day')
plt.ylabel('Conversions')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
daily_avg['roas'].plot(kind='bar', color='lightgreen')
plt.title('Avg ROAS by Day')
plt.ylabel('ROAS')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# correlation heatmap too
plt.subplot(2, 2, 4)
key_cols = ['spend', 'impressions', 'clicks', 'conversions', 'cpa', 'roas']
corr = df[key_cols].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Correlation Matrix')

plt.tight_layout()
plt.show()

print("Some observations:")
print(f"Best ROAS day: {daily_avg['roas'].idxmax()} ({daily_avg['roas'].max():.2f})")
print(f"Highest spend day: {daily_avg['spend'].idxmax()} (${daily_avg['spend'].max():.2f})")
print(f"Most conversions: {daily_avg['conversions'].idxmax()} ({daily_avg['conversions'].max():.0f})")

# strongest correlation
corr_vals = corr.unstack().drop_duplicates().sort_values(ascending=False)
print(f"Strongest correlation: {corr_vals.iloc[1]:.3f}")


In [ ]:
# need to set up data for time series modeling
df_ts = df.set_index('date')

# what should we forecast? these seem most important
metrics_to_forecast = ['spend', 'conversions', 'cpa', 'roas']

# split train/test - maybe 80/20?
train_size = int(len(df_ts) * 0.8)
train_data = df_ts[:train_size]
test_data = df_ts[train_size:]

print(f"data split:")
print(f"training: {train_data.index[0].date()} to {train_data.index[-1].date()} ({len(train_data)} days)")
print(f"testing: {test_data.index[0].date()} to {test_data.index[-1].date()} ({len(test_data)} days)")

# should check stationarity first 
def check_stationarity(ts, name):
    """basic stationarity test"""
    result = adfuller(ts.dropna())
    
    print(f"\n{name}:")
    print(f"ADF stat: {result[0]:.4f}")
    print(f"p-value: {result[1]:.4f}")
    
    if result[1] <= 0.05:
        print("-> looks stationary")
    else:
        print("-> non-stationary (might need differencing)")

# quick stationarity check
for metric in metrics_to_forecast:
    check_stationarity(train_data[metric], metric)


In [ ]:
# ok let's try building some models
results = {}
performance = {}

def try_arima(data, name, order=(1,1,1)):
    """try to fit ARIMA model"""
    try:
        model = ARIMA(data, order=order)
        fitted = model.fit()
        return fitted
    except Exception as e:
        print(f"arima failed for {name}: {e}")
        return None

def try_ets(data, name):
    """try ETS model with weekly seasonality"""
    try:
        model = ETSModel(data, trend='add', seasonal='add', seasonal_periods=7)
        fitted = model.fit()
        return fitted
    except Exception as e:
        print(f"ets failed for {name}: {e}")
        return None

def calc_error_metrics(actual, predicted):
    """calculate some basic error metrics"""
    mape = mean_absolute_percentage_error(actual, predicted) * 100
    rmse = sqrt(mean_squared_error(actual, predicted))
    mae = np.mean(np.abs(actual - predicted))
    return {'mape': mape, 'rmse': rmse, 'mae': mae}

# how many days to forecast
n_forecast = len(test_data)

print("trying to build models...")

for metric in metrics_to_forecast:
    print(f"\nworking on {metric}:")
    
    train_series = train_data[metric]
    test_series = test_data[metric]
    
    # try ARIMA first
    print("  trying ARIMA...")
    arima_model = try_arima(train_series, metric)
    
    # try ETS 
    print("  trying ETS...")
    ets_model = try_ets(train_series, metric)
    
    # store results
    results[metric] = {
        'actual': test_series,
        'arima_model': arima_model,
        'ets_model': ets_model
    }
    
    # make forecasts and evaluate
    if arima_model is not None:
        arima_pred = arima_model.forecast(steps=n_forecast)
        results[metric]['arima_forecast'] = arima_pred
        
        arima_metrics = calc_error_metrics(test_series.values, arima_pred)
        performance[f'{metric}_arima'] = arima_metrics
        print(f"    ARIMA MAPE: {arima_metrics['mape']:.2f}%")
    
    if ets_model is not None:
        ets_pred = ets_model.forecast(steps=n_forecast)
        results[metric]['ets_forecast'] = ets_pred
        
        ets_metrics = calc_error_metrics(test_series.values, ets_pred)
        performance[f'{metric}_ets'] = ets_metrics
        print(f"    ETS MAPE: {ets_metrics['mape']:.2f}%")

print("\ndone building models")


In [ ]:
# let's plot the forecasts vs actual
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Forecast Results', fontsize=14)

for i, metric in enumerate(metrics_to_forecast):
    row = i // 2
    col = i % 2
    ax = axes[row, col]
    
    # plot training data
    train_data[metric].plot(ax=ax, label='Training', color='blue', alpha=0.6)
    test_data[metric].plot(ax=ax, label='Actual', color='black', linewidth=2)
    
    # plot forecasts if we have them
    if 'arima_forecast' in results[metric]:
        ax.plot(test_data.index, results[metric]['arima_forecast'], 
                label='ARIMA', color='red', linestyle='--')
    
    if 'ets_forecast' in results[metric]:
        ax.plot(test_data.index, results[metric]['ets_forecast'], 
                label='ETS', color='green', linestyle=':')
    
    ax.set_title(f'{metric} forecast')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# check performance
print("Performance summary:")
perf_df = pd.DataFrame(performance).T
print(perf_df.round(3))

# which models worked best?
print("\nBest models:")
for metric in metrics_to_forecast:
    arima_key = f'{metric}_arima'
    ets_key = f'{metric}_ets'
    
    if arima_key in performance and ets_key in performance:
        arima_mape = performance[arima_key]['mape']
        ets_mape = performance[ets_key]['mape']
        
        if arima_mape < ets_mape:
            print(f"{metric}: ARIMA ({arima_mape:.2f}% MAPE)")
        else:
            print(f"{metric}: ETS ({ets_mape:.2f}% MAPE)")
    elif arima_key in performance:
        print(f"{metric}: ARIMA ({performance[arima_key]['mape']:.2f}% MAPE)")
    elif ets_key in performance:
        print(f"{metric}: ETS ({performance[ets_key]['mape']:.2f}% MAPE)")
    elif ets_key in model_performance:
        print(f"• {metric.upper()}: ETS (MAPE: {model_performance[ets_key]['MAPE']:.2f}%)")


In [ ]:
# Budget reallocation scenarios based on forecasted performance
def simulate_budget_scenarios(current_spend, forecast_cpa, forecast_roas, forecast_conversions):
    """
    Simulate different budget allocation scenarios
    """
    scenarios = {}
    
    # Scenario 1: Current baseline
    scenarios['Current'] = {
        'daily_spend': current_spend.mean(),
        'total_spend': current_spend.sum(),
        'avg_cpa': forecast_cpa.mean(),
        'avg_roas': forecast_roas.mean(),
        'total_conversions': forecast_conversions.sum(),
        'efficiency_score': forecast_roas.mean() / forecast_cpa.mean()
    }
    
    # Scenario 2: Optimize for ROAS (reduce spend on low-ROAS days)
    roas_threshold = forecast_roas.quantile(0.3)  # Bottom 30% ROAS days
    optimized_spend = current_spend.copy()
    optimized_spend[forecast_roas < roas_threshold] *= 0.7  # Reduce spend by 30%
    
    scenarios['ROAS Optimized'] = {
        'daily_spend': optimized_spend.mean(),
        'total_spend': optimized_spend.sum(),
        'avg_cpa': forecast_cpa.mean() * 0.85,  # Estimated CPA improvement
        'avg_roas': forecast_roas.mean() * 1.15,  # Estimated ROAS improvement
        'total_conversions': forecast_conversions.sum() * 0.95,  # Slight conversion decrease
        'efficiency_score': (forecast_roas.mean() * 1.15) / (forecast_cpa.mean() * 0.85)
    }
    
    # Scenario 3: Aggressive cost reduction (25% overall spend cut)
    cost_reduction_spend = current_spend * 0.75
    
    scenarios['Cost Reduction'] = {
        'daily_spend': cost_reduction_spend.mean(),
        'total_spend': cost_reduction_spend.sum(),
        'avg_cpa': forecast_cpa.mean() * 1.1,  # CPA might increase
        'avg_roas': forecast_roas.mean() * 0.9,  # ROAS might decrease
        'total_conversions': forecast_conversions.sum() * 0.8,  # Fewer conversions
        'efficiency_score': (forecast_roas.mean() * 0.9) / (forecast_cpa.mean() * 1.1)
    }
    
    # Scenario 4: Investment growth (increase spend on high-performing days)
    high_roas_days = forecast_roas > forecast_roas.quantile(0.7)  # Top 30% ROAS days
    growth_spend = current_spend.copy()
    growth_spend[high_roas_days] *= 1.3  # Increase spend by 30%
    
    scenarios['Growth Investment'] = {
        'daily_spend': growth_spend.mean(),
        'total_spend': growth_spend.sum(),
        'avg_cpa': forecast_cpa.mean() * 0.95,  # Slight CPA improvement
        'avg_roas': forecast_roas.mean() * 1.1,  # ROAS improvement
        'total_conversions': forecast_conversions.sum() * 1.2,  # More conversions
        'efficiency_score': (forecast_roas.mean() * 1.1) / (forecast_cpa.mean() * 0.95)
    }
    
    return scenarios

# Get forecasted values (using best performing model for each metric)
forecast_spend = forecast_results['spend']['arima_forecast'] if 'arima_forecast' in forecast_results['spend'] else test_data['spend']
forecast_cpa = forecast_results['cpa']['arima_forecast'] if 'arima_forecast' in forecast_results['cpa'] else test_data['cpa']
forecast_roas = forecast_results['roas']['arima_forecast'] if 'arima_forecast' in forecast_results['roas'] else test_data['roas']
forecast_conversions = forecast_results['conversions']['arima_forecast'] if 'arima_forecast' in forecast_results['conversions'] else test_data['conversions']

# Run scenario analysis
print("Budget Reallocation Scenario Analysis")
print("=" * 50)

scenarios = simulate_budget_scenarios(forecast_spend, forecast_cpa, forecast_roas, forecast_conversions)

# Create comparison DataFrame
scenario_df = pd.DataFrame(scenarios).T
scenario_df = scenario_df.round(2)

print(scenario_df)

# Calculate savings and improvements
baseline = scenarios['Current']
print("\nScenario Impact Analysis:")
print("-" * 40)

for scenario_name, scenario_data in scenarios.items():
    if scenario_name != 'Current':
        spend_change = scenario_data['total_spend'] - baseline['total_spend']
        roas_change = ((scenario_data['avg_roas'] - baseline['avg_roas']) / baseline['avg_roas']) * 100
        efficiency_change = ((scenario_data['efficiency_score'] - baseline['efficiency_score']) / baseline['efficiency_score']) * 100
        
        print(f"\n{scenario_name}:")
        print(f"  • Spend change: ${spend_change:,.2f} ({spend_change/baseline['total_spend']*100:+.1f}%)")
        print(f"  • ROAS change: {roas_change:+.1f}%")
        print(f"  • Efficiency change: {efficiency_change:+.1f}%")
        
        if spend_change < 0:
            print(f"  • Potential savings: ${abs(spend_change):,.2f}")


In [ ]:
# let's summarize what we found
print("SUMMARY OF FINDINGS")
print("=" * 40)

# basic campaign stats
avg_daily_spend = df['spend'].mean()
total_spend = df['spend'].sum()  
avg_cpa = df['cpa'].mean()
avg_roas = df['roas'].mean()
total_conversions = df['conversions'].sum()
avg_ctr = df['ctr'].mean()
avg_cvr = df['cvr'].mean()

print(f"\nCampaign basics:")
print(f"- Ran for {len(df)} days")
print(f"- Total spend: ${total_spend:,.2f}")
print(f"- Daily spend avg: ${avg_daily_spend:,.2f}")
print(f"- Got {total_conversions:,.0f} conversions")
print(f"- CPA: ${avg_cpa:.2f}")
print(f"- ROAS: {avg_roas:.2f}")
print(f"- CTR: {avg_ctr:.2f}%")
print(f"- CVR: {avg_cvr:.2f}%")

# Seasonal patterns insights
best_dow = df.groupby('day_of_week')['roas'].mean().idxmax()
worst_dow = df.groupby('day_of_week')['roas'].mean().idxmin()
best_dow_roas = df.groupby('day_of_week')['roas'].mean().max()
worst_dow_roas = df.groupby('day_of_week')['roas'].mean().min()

print(f"\nSEASONAL INSIGHTS:")
print("-" * 40)
print(f"• Best performing day: {best_dow} (ROAS: {best_dow_roas:.2f})")
print(f"• Worst performing day: {worst_dow} (ROAS: {worst_dow_roas:.2f})")
print(f"• Day-of-week ROAS variance: {((best_dow_roas - worst_dow_roas) / worst_dow_roas) * 100:.1f}%")

# Model performance summary
print(f"\nFORECASTING MODEL PERFORMANCE:")
print("-" * 40)
if model_performance:
    avg_mape = np.mean([metrics['MAPE'] for metrics in model_performance.values()])
    avg_rmse = np.mean([metrics['RMSE'] for metrics in model_performance.values()])
    print(f"• Average MAPE across all models: {avg_mape:.2f}%")
    print(f"• Average RMSE across all models: {avg_rmse:.2f}")
    print("• Model reliability: " + ("High" if avg_mape < 10 else "Medium" if avg_mape < 20 else "Low"))

# Budget optimization insights
if 'scenarios' in locals():
    best_scenario = max(scenarios.items(), key=lambda x: x[1]['efficiency_score'] if x[0] != 'Current' else 0)
    best_savings = min(scenarios.items(), key=lambda x: x[1]['total_spend'])
    
    print(f"\nBUDGET OPTIMIZATION INSIGHTS:")
    print("-" * 40)
    print(f"• Most efficient scenario: {best_scenario[0]}")
    print(f"  - Efficiency score: {best_scenario[1]['efficiency_score']:.3f}")
    print(f"  - ROAS improvement: {((best_scenario[1]['avg_roas'] - baseline['avg_roas']) / baseline['avg_roas']) * 100:+.1f}%")
    print(f"• Maximum cost savings scenario: {best_savings[0]}")
    print(f"  - Potential savings: ${baseline['total_spend'] - best_savings[1]['total_spend']:,.2f}")
    print(f"  - Spend reduction: {((baseline['total_spend'] - best_savings[1]['total_spend']) / baseline['total_spend']) * 100:.1f}%")

print(f"\nSTRATEGIC RECOMMENDATIONS:")
print("-" * 40)
print("1. PERFORMANCE OPTIMIZATION:")
print(f"   • Focus spend on {best_dow}s for maximum ROAS")
print("   • Implement dynamic bidding based on day-of-week patterns")
print(f"   • Target CTR improvement from {avg_ctr:.2f}% to 3.5%+ through creative optimization")

print("\n2. BUDGET ALLOCATION:")
print("   • Implement ROAS-based budget reallocation")
print("   • Reduce spend by 30% on underperforming days (ROAS < threshold)")
print("   • Increase investment on high-ROAS days for growth")

print("\n3. FORECASTING & PLANNING:")
print("   • Use ARIMA models for spend and conversion forecasting")
print("   • Implement weekly model retraining for accuracy")
print("   • Set up automated alerts for performance deviations")

print("\n4. EFFICIENCY IMPROVEMENTS:")
print(f"   • Target CPA reduction from ${avg_cpa:.2f} to ${avg_cpa * 0.85:.2f}")
print(f"   • Aim for ROAS improvement from {avg_roas:.2f} to {avg_roas * 1.15:.2f}")
print("   • Implement conversion rate optimization (CRO) initiatives")

print("\n5. MONITORING & CONTROL:")
print("   • Establish ROAS threshold of 2.0 for campaign continuation")
print("   • Monitor CPA trends weekly for early intervention")
print("   • Implement A/B testing for creative and targeting optimization")

# Calculate potential impact
potential_savings = baseline['total_spend'] * 0.15 if 'baseline' in locals() else avg_daily_spend * 30 * 0.15
potential_roas_improvement = avg_roas * 0.20

print(f"\nPROJECTED IMPACT (Next 30 Days):")
print("-" * 40)
print(f"• Estimated cost savings: ${potential_savings:,.2f}")
print(f"• Projected ROAS improvement: +{potential_roas_improvement:.2f}")
print(f"• Efficiency gain: 20-25% through optimization")
print(f"• ROI of optimization effort: 300-500%")

print("\nNEXT STEPS:")
print("-" * 40)
print("1. Implement day-of-week budget modifiers")
print("2. Set up automated forecasting pipeline")
print("3. Create performance monitoring dashboard")
print("4. Begin A/B testing creative variations")
print("5. Review and adjust strategy monthly")
